In [1]:
import json
import os
import re
import numpy as np
import math
from collections import defaultdict, Counter

# Data Loading and Configuration

In [2]:
# Define paths
DATA_DIR = "IR-03-1-HW1-Data"
TRAIN_PATH = os.path.join(DATA_DIR, "train")
VAL_PATH = os.path.join(DATA_DIR, "val")
TEST_PATH = os.path.join(DATA_DIR, "test")

def load_json_data(path, file_type):
    """
    Loads data based on file type (passages, questions, judg).
    """
    file_map = {
        "passages": "passages.json",
        "questions": "questions.json",
        "judg": "Judg.json"
    }
    
    full_path = os.path.join(path, file_map[file_type])
    
    if not os.path.exists(full_path):
        print(f"File not found: {full_path}")
        return {}

    with open(full_path, 'r', encoding='utf-8') as f:
        data = json.load(f)

    # Parse based on expected structure
    if file_type == "passages":
        # Returns dict: {doc_id: text}
        return {str(item['doc_id']): item['passage_text'] for item in data}
    
    elif file_type == "questions":
        # Returns dict: {query_id: text}
        return {str(item['query_id']): item['query_text'] for item in data}
    
    elif file_type == "judg":
        # Returns dict: {query_id: [doc_id1, doc_id2, ...]}
        clean_judg = {}
        for qid, docs in data.items():
            # Handle string format "doc1, doc2" seen in snippets
            if isinstance(docs, str):
                clean_judg[str(qid)] = [d.strip() for d in docs.split(',')]
            elif isinstance(docs, list):
                clean_judg[str(qid)] = [str(d) for d in docs]
        return clean_judg

print("Loading Data:")
# Load Train (Only passages available per PDF)
train_passages = load_json_data(TRAIN_PATH, "passages")

# Load Validation
val_passages = load_json_data(VAL_PATH, "passages")
val_questions = load_json_data(VAL_PATH, "questions")
val_judg = load_json_data(VAL_PATH, "judg")

# Load Test
test_passages = load_json_data(TEST_PATH, "passages")
test_questions = load_json_data(TEST_PATH, "questions")
test_judg = load_json_data(TEST_PATH, "judg")

print(f"Train Docs: {len(train_passages)}")
print(f"Val Docs: {len(val_passages)}, Val Queries: {len(val_questions)}")
print(f"Test Docs: {len(test_passages)}, Test Queries: {len(test_questions)}")

Loading Data:
Train Docs: 750
Val Docs: 1245, Val Queries: 20
Test Docs: 1245, Test Queries: 30


# Preprocessing and Vocabulary (At least top 500)

In [3]:
# Simple stopword list for English:
STOPWORDS = set([
    'the', 'is', 'at', 'which', 'on', 'and', 'a', 'an', 'in', 'to', 'of', 'for', 'it', 'that', 'with', 
    'as', 'are', 'by', 'this', 'be', 'or', 'from', 'but', 'not', 'can', 'have', 'has', 'was', 'were',
    'what', 'how', 'does', 'do', 'why', 'where', 'when', 'who'
])

def preprocess(text):
    """
    Lowercases, removes punctuation, tokenizes, and removes stopwords.
    """
    if not isinstance(text, str): return []
    text = text.lower()
    # Remove punctuation
    text = re.sub(r'[^a-z0-9\s]', ' ', text)
    tokens = text.split()
    return [t for t in tokens if t not in STOPWORDS]

def build_vocab(documents, top_k=500):
    """
    Builds vocabulary from TRAINING documents using top K frequent words.
    Constraint: Length of TF-IDF vector must be at least 500.
    """
    print(f"Building vocabulary from {len(documents)} training documents:")
    all_tokens = []
    for doc_text in documents.values():
        all_tokens.extend(preprocess(doc_text))
    
    # Count frequencies
    counts = Counter(all_tokens)
    
    # Select top K (K must be at least 500)
    most_common = counts.most_common(top_k)
    vocab = {term for term, freq in most_common}
    
    return vocab, counts

# Build Vocab from TRAINING data:
VOCAB, TRAIN_COUNTS = build_vocab(train_passages, top_k=500)
print(f"Vocabulary size: {len(VOCAB)}")

def filter_query(query_text):
    """
    Preprocesses query and filters words not in VOCAB.
    Constraint: New words should not be considered.
    """
    tokens = preprocess(query_text)
    return [t for t in tokens if t in VOCAB]

Building vocabulary from 750 training documents:
Vocabulary size: 500


# Evaluation metrics

In [4]:
# Constraints: Implement manually, no libraries:

def calculate_metrics(retrieved_doc_ids, relevant_doc_ids, k=5):
    """
    Calculates P@K, MRR, and MAP for a single query.
    """
    relevant_set = set(relevant_doc_ids)
    
    # 1. Precision at K (P@5)
    retrieved_k = retrieved_doc_ids[:k]
    hits_k = sum(1 for doc in retrieved_k if doc in relevant_set)
    p_at_k = hits_k / k
    
    # 2. Mean Reciprocal Rank (MRR)
    mrr = 0.0
    for i, doc in enumerate(retrieved_doc_ids):
        if doc in relevant_set:
            mrr = 1.0 / (i + 1)
            break
            
    # 3. Average Precision (AP)
    # Constraint: Denominator = number of relevant documents
    hits = 0
    sum_prec = 0.0
    for i, doc in enumerate(retrieved_doc_ids):
        if doc in relevant_set:
            hits += 1
            sum_prec += hits / (i + 1)
            
    if len(relevant_set) > 0:
        ap = sum_prec / len(relevant_set)
    else:
        ap = 0.0
        
    return p_at_k, mrr, ap

def evaluate_model(model, queries, judgments, dataset_passages, top_k_retrieval=50):
    """
    Runs search for all queries and computes average P@5, MRR, MAP.
    """
    avg_p5 = 0.0
    avg_mrr = 0.0
    avg_map = 0.0
    n = 0
    
    # To be sure that model is indexed on the correct dataset
    model.index_documents(dataset_passages)
    
    for qid, q_text in queries.items():
        if qid not in judgments: continue
        
        # Retrieve docs
        results = model.search(q_text, top_k=top_k_retrieval)
        retrieved_ids = [doc_id for doc_id, score in results]
        relevant_ids = judgments[qid]
        
        p5, mrr, ap = calculate_metrics(retrieved_ids, relevant_ids, k=5)
        
        avg_p5 += p5
        avg_mrr += mrr
        avg_map += ap
        n += 1
        
    if n == 0: return 0, 0, 0
    return avg_p5/n, avg_mrr/n, avg_map/n

# Section Two: Retrieval Using the BM25 Model

In [5]:
class BM25:
    def __init__(self, vocab, k1=1.5, b=0.75):
        self.vocab = vocab
        self.k1 = k1
        self.b = b
        self.inverted_index = defaultdict(list)
        self.doc_lengths = {}
        self.avg_dl = 0
        self.doc_count = 0
        self.idf = {}

    def index_documents(self, documents):
        """Builds index for the given document set (Val or Test)."""
        self.inverted_index = defaultdict(list)
        self.doc_lengths = {}
        self.doc_count = len(documents)
        total_len = 0
        doc_freqs = Counter()

        for doc_id, text in documents.items():
            # Filter tokens by VOCAB 
            tokens = [t for t in preprocess(text) if t in self.vocab]
            length = len(tokens)
            self.doc_lengths[doc_id] = length
            total_len += length
            
            # Term Frequencies in Doc
            term_counts = Counter(tokens)
            for term, count in term_counts.items():
                self.inverted_index[term].append((doc_id, count))
                doc_freqs[term] += 1
        
        self.avg_dl = total_len / self.doc_count if self.doc_count > 0 else 0
        
        # Precompute IDF
        for term in self.vocab:
            df = doc_freqs.get(term, 0)
            # Standard Probabilistic IDF
            self.idf[term] = math.log((self.doc_count - df + 0.5) / (df + 0.5) + 1)

    def search(self, query, top_k=5):
        tokens = filter_query(query)
        scores = defaultdict(float)
        
        for term in tokens:
            if term not in self.inverted_index: continue
            
            idf_val = self.idf[term]
            
            for doc_id, tf in self.inverted_index[term]:
                doc_len = self.doc_lengths[doc_id]
                
                # BM25 Formula
                num = tf * (self.k1 + 1)
                den = tf + self.k1 * (1 - self.b + self.b * (doc_len / self.avg_dl))
                
                scores[doc_id] += idf_val * (num / den)
        
        return sorted(scores.items(), key=lambda x: x[1], reverse=True)[:top_k]

# Section Three: Retrieval Using Language Models
1. Unigram Language Model
2. Bigram Language Model 

In [ ]:
class UnigramLM:
    def __init__(self, vocab, train_counts, mu=1000):
        self.vocab = vocab
        self.mu = mu
        self.inverted_index = defaultdict(list)
        self.doc_lengths = {}
        
        # Compute Collection Probability P(w|C) from Training Data
        total_train_tokens = sum(train_counts.values())
        self.p_C = {t: c / total_train_tokens for t, c in train_counts.items()}

    def index_documents(self, documents):
        self.inverted_index = defaultdict(list)
        self.doc_lengths = {}
        
        for doc_id, text in documents.items():
            tokens = [t for t in preprocess(text) if t in self.vocab]
            self.doc_lengths[doc_id] = len(tokens)
            
            counts = Counter(tokens)
            for term, count in counts.items():
                self.inverted_index[term].append((doc_id, count))

    def search(self, query, top_k=5):
        tokens = filter_query(query)
        scores = defaultdict(float)
        
        # Only score documents that contain at least one query term (Optimization)
        candidate_docs = set()
        for term in tokens:
            for doc_id, _ in self.inverted_index[term]:
                candidate_docs.add(doc_id)
        
        for doc_id in candidate_docs:
            doc_len = self.doc_lengths[doc_id]
            log_prob = 0.0
            
            for term in tokens:
                # Find TF in current doc (naive search in list)
                tf = 0
                for d, c in self.inverted_index[term]:
                    if d == doc_id:
                        tf = c
                        break
                
                # Dirichlet Smoothing
                p_wc = self.p_C.get(term, 1e-10)
                numerator = tf + (self.mu * p_wc)
                denominator = doc_len + self.mu
                
                prob = numerator / denominator
                log_prob += math.log(prob)
            
            scores[doc_id] = log_prob

        return sorted(scores.items(), key=lambda x: x[1], reverse=True)[:top_k]

class BigramLM:
    def __init__(self, vocab, train_counts, unigram_model, lam=0.5):
        self.vocab = vocab
        self.unigram_model = unigram_model
        self.lam = lam # Acting as the lambda for Unigram weight
        self.chi = 1.0 - lam # Weight for Bigram
        self.doc_tokens = {} # Store full token lists for bigram counting
        
    def index_documents(self, documents):
        self.doc_tokens = {}
        # Need the unigram model to be indexed on the same docs for the smoothing part
        self.unigram_model.index_documents(documents)
        
        for doc_id, text in documents.items():
            self.doc_tokens[doc_id] = [t for t in preprocess(text) if t in self.vocab]

    def search(self, query, top_k=5):
        tokens = filter_query(query)
        if not tokens: return []
        
        scores = defaultdict(float)
        # Use unigram candidates
        candidate_docs = self.unigram_model.doc_lengths.keys()
        
        for doc_id in candidate_docs:
            doc_seq = self.doc_tokens.get(doc_id, [])
            doc_len = len(doc_seq)
            if doc_len == 0: continue
            
            log_prob = 0.0
            
            # 1. First term: P(q1 | D) using Smoothed Unigram
            q1 = tokens[0]
            tf_q1 = doc_seq.count(q1)
            p_uni_q1 = (tf_q1 + self.unigram_model.mu * self.unigram_model.p_C.get(q1, 1e-10)) / (doc_len + self.unigram_model.mu)
            log_prob += math.log(p_uni_q1)
            
            # 2. Subsequent terms: Bigram P(qi | qi-1, D)
            for i in range(1, len(tokens)):
                qi = tokens[i]
                q_prev = tokens[i-1]
                
                # Count Bigram C(qi, qi-1, D) and Unigram C(qi-1, D)
                c_bigram = 0
                for j in range(len(doc_seq) - 1):
                    if doc_seq[j] == q_prev and doc_seq[j+1] == qi:
                        c_bigram += 1
                
                c_prev = doc_seq.count(q_prev)
                
                # Empirical Bigram Probability
                p_bigram_emp = (c_bigram / c_prev) if c_prev > 0 else 0.0
                
                # Smoothed Unigram Probability for qi
                tf_qi = doc_seq.count(qi)
                p_uni_smooth = (tf_qi + self.unigram_model.mu * self.unigram_model.p_C.get(qi, 1e-10)) / (doc_len + self.unigram_model.mu)
                
                # Interpolation 
                final_prob = (self.chi * p_bigram_emp) + (self.lam * p_uni_smooth)
                
                if final_prob <= 0: final_prob = 1e-10
                log_prob += math.log(final_prob)
                
            scores[doc_id] = log_prob
            
        return sorted(scores.items(), key=lambda x: x[1], reverse=True)[:top_k]

# Section Four: Execution, tuning and evaluation of results

In [7]:
print("\n--- Phase 1: Tuning on Validation Dataset ---")

# A. Tune BM25
# Constraints: Test at least 3 values.
k1_vals = [1.2, 1.5, 2.0]
b_vals = [0.5, 0.75, 1.0]
best_bm25_map = -1
best_bm25_params = (1.5, 0.75)

for k in k1_vals:
    for b in b_vals:
        model = BM25(VOCAB, k1=k, b=b)
        _, _, map_score = evaluate_model(model, val_questions, val_judg, val_passages)
        print(f"BM25 (k1={k}, b={b}) -> MAP: {map_score:.4f}")
        if map_score > best_bm25_map:
            best_bm25_map = map_score
            best_bm25_params = (k, b)

print(f"Best BM25 Params: {best_bm25_params}")

# B. Tune Unigram LM
# Constraint: Find appropriate mu.
mu_vals = [500, 1000, 2000]
best_uni_map = -1
best_mu = 1000

for mu in mu_vals:
    model = UnigramLM(VOCAB, TRAIN_COUNTS, mu=mu)
    _, _, map_score = evaluate_model(model, val_questions, val_judg, val_passages)
    print(f"Unigram (mu={mu}) -> MAP: {map_score:.4f}")
    if map_score > best_uni_map:
        best_uni_map = map_score
        best_mu = mu

print(f"Best Unigram Mu: {best_mu}")

# C. Tune Bigram LM
# Constraint: Find optimal lambda.
lambda_vals = [0.1, 0.5, 0.9]
best_bi_map = -1
best_lam = 0.5

# Note: We reuse the best Mu found above for the unigram component
base_unigram = UnigramLM(VOCAB, TRAIN_COUNTS, mu=best_mu)

for lam in lambda_vals:
    model = BigramLM(VOCAB, TRAIN_COUNTS, base_unigram, lam=lam)
    # Re-indexing handled inside evaluate_model -> model.index_documents
    _, _, map_score = evaluate_model(model, val_questions, val_judg, val_passages)
    print(f"Bigram (lambda={lam}) -> MAP: {map_score:.4f}")
    if map_score > best_bi_map:
        best_bi_map = map_score
        best_lam = lam

print(f"Best Bigram Lambda: {best_lam}")


print("\n--- Phase 2: Final Evaluation on Test Dataset ---")
# Constraint: Evaluate on Test using best params.

# 1. BM25 Final
bm25_final = BM25(VOCAB, k1=best_bm25_params[0], b=best_bm25_params[1])
p5, mrr, map_ = evaluate_model(bm25_final, test_questions, test_judg, test_passages)
print(f"BM25 TEST RESULTS -> P@5: {p5:.4f}, MRR: {mrr:.4f}, MAP: {map_:.4f}")

# 2. Unigram Final
uni_final = UnigramLM(VOCAB, TRAIN_COUNTS, mu=best_mu)
p5, mrr, map_ = evaluate_model(uni_final, test_questions, test_judg, test_passages)
print(f"Unigram TEST RESULTS -> P@5: {p5:.4f}, MRR: {mrr:.4f}, MAP: {map_:.4f}")

# 3. Bigram Final
base_uni_final = UnigramLM(VOCAB, TRAIN_COUNTS, mu=best_mu)
bi_final = BigramLM(VOCAB, TRAIN_COUNTS, base_uni_final, lam=best_lam)
p5, mrr, map_ = evaluate_model(bi_final, test_questions, test_judg, test_passages)
print(f"Bigram TEST RESULTS -> P@5: {p5:.4f}, MRR: {mrr:.4f}, MAP: {map_:.4f}")


--- Phase 1: Tuning on Validation Dataset ---
BM25 (k1=1.2, b=0.5) -> MAP: 0.3199
BM25 (k1=1.2, b=0.75) -> MAP: 0.3210
BM25 (k1=1.2, b=1.0) -> MAP: 0.3158
BM25 (k1=1.5, b=0.5) -> MAP: 0.3163
BM25 (k1=1.5, b=0.75) -> MAP: 0.3211
BM25 (k1=1.5, b=1.0) -> MAP: 0.3177
BM25 (k1=2.0, b=0.5) -> MAP: 0.3151
BM25 (k1=2.0, b=0.75) -> MAP: 0.3175
BM25 (k1=2.0, b=1.0) -> MAP: 0.3103
Best BM25 Params: (1.5, 0.75)
Unigram (mu=500) -> MAP: 0.2934
Unigram (mu=1000) -> MAP: 0.2916
Unigram (mu=2000) -> MAP: 0.2899
Best Unigram Mu: 500
Bigram (lambda=0.1) -> MAP: 0.3091
Bigram (lambda=0.5) -> MAP: 0.3091
Bigram (lambda=0.9) -> MAP: 0.3116
Best Bigram Lambda: 0.9

--- Phase 2: Final Evaluation on Test Dataset ---
BM25 TEST RESULTS -> P@5: 0.2800, MRR: 0.4648, MAP: 0.2198
Unigram TEST RESULTS -> P@5: 0.2667, MRR: 0.3744, MAP: 0.1927
Bigram TEST RESULTS -> P@5: 0.2933, MRR: 0.4295, MAP: 0.2156
